In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rtatman/handwritten-mathematical-expressions")

print("Path to dataset files:", path)

c:\Users\MSI\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 65.9M/65.9M [00:12<00:00, 5.76MB/s]

Extracting files...


Path to dataset files: C:\Users\MSI\.cache\kagglehub\datasets\rtatman\handwritten-mathematical-expressions\versions\1


In [ ]:
# Chạy trực tiếp trong notebook
import sys
sys.path.append('.')

# Import các hàm từ test.py
from test.test import visualize_inkml
from pathlib import Path

# Chọn file inkml để hiển thị
inkml_file = Path("handwritten-mathematical-expressions/versions/1/MatricesTest2014/MatricesTest/RIT_MatrixTest_2014_2.inkml")

# Kiểm tra file tồn tại
if inkml_file.exists():
    visualize_inkml(inkml_file)
else:
    print(f"Không tìm thấy file: {inkml_file}")


In [4]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

def parse_trace(trace_text: str):
    """
    Chuyển chuỗi 'x y, x y, …' thành list[(x, y)]
    """
    points = []
    for pair in trace_text.strip().split(','):
        if pair.strip():
            x_str, y_str = pair.strip().split()
            points.append((float(x_str), float(y_str)))
    return points

def visualize_inkml(file_path: str,
                    flip_y: bool = True,
                    figsize=(10, 8),
                    show_latex=True,
                    show_segments=True):
    """
    Đọc & hiển thị nét vẽ của một file .inkml
    ------------------------------------------------
    file_path : đường dẫn tới file inkml
    flip_y    : True → lật trục Y cho phù hợp hệ tọa độ matplotlib
    figsize   : kích thước figure
    show_latex : hiển thị LaTeX ground-truth dưới dạng text (không render)
    show_segments : hiển thị thông tin phân đoạn (symbol segmentation)
    """
    try:
        file_path = Path(file_path)
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Lấy LaTeX ground-truth (annotation type="truth")
        latex = None
        for ann in root.findall('.//{http://www.w3.org/2003/InkML}annotation'):
            if ann.get('type') == 'truth':
                latex = ann.text.strip() if ann.text else ''
                break

        # Thu thập & vẽ tất cả trace
        fig, ax = plt.subplots(figsize=figsize)
        
        # Tính toán bounding box cho tất cả traces
        all_x = []
        all_y = []
        
        traces = root.findall('.//{http://www.w3.org/2003/InkML}trace')
        
        # Lấy thông tin segmentation nếu có
        segments = {}
        if show_segments:
            seg_group = root.find('.//{http://www.w3.org/2003/InkML}traceGroup[@xml:id="-- Segmentation Data --"]')
            if seg_group is not None:
                for tg in seg_group.findall('.//{http://www.w3.org/2003/InkML}traceGroup'):
                    symbol_id = tg.get('{http://www.w3.org/XML/1998/namespace}id', '')
                    symbol_truth = ''
                    for ann in tg.findall('.//{http://www.w3.org/2003/InkML}annotation'):
                        if ann.get('type') == 'truth':
                            symbol_truth = ann.text if ann.text else ''
                            break
                    
                    trace_views = tg.findall('.//{http://www.w3.org/2003/InkML}traceView')
                    for tv in trace_views:
                        trace_ref = tv.get('traceDataRef', '')
                        if trace_ref:
                            segments[trace_ref] = {'symbol': symbol_truth, 'id': symbol_id}
        
        # Màu sắc khác nhau cho từng trace
        colors = plt.cm.tab10(np.linspace(0, 1, len(traces)))
        
        for idx, tr in enumerate(traces):
            trace_id = tr.get('id', str(idx))
            pts = parse_trace(tr.text)
            if not pts:
                continue
            xs, ys = zip(*pts)
            if flip_y:
                # Lật trục y (InkML gốc y tăng xuống; matplotlib y tăng lên)
                ys = [-y for y in ys]
            
            # Label cho trace
            label = f'Trace {trace_id}'
            if trace_id in segments:
                label = f"'{segments[trace_id]['symbol']}' (trace {trace_id})"
            
            # Vẽ từng nét với màu khác nhau
            ax.plot(xs, ys, linewidth=3, color=colors[idx % len(colors)], 
                    marker='o', markersize=2, alpha=0.8, label=label)
            
            # Đánh dấu điểm đầu của mỗi trace
            ax.plot(xs[0], ys[0], 'o', color=colors[idx % len(colors)], 
                    markersize=8, markeredgecolor='black', markeredgewidth=1)
            
            all_x.extend(xs)
            all_y.extend(ys)

        # Thiết lập giới hạn trục với padding
        if all_x and all_y:
            x_margin = (max(all_x) - min(all_x)) * 0.1
            y_margin = (max(all_y) - min(all_y)) * 0.1
            ax.set_xlim(min(all_x) - x_margin, max(all_x) + x_margin)
            ax.set_ylim(min(all_y) - y_margin, max(all_y) + y_margin)
        
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        ax.axhline(y=0, color='k', linewidth=0.5, alpha=0.3)
        ax.axvline(x=0, color='k', linewidth=0.5, alpha=0.3)
        
        # Tiêu đề - an toàn không render LaTeX
        title_lines = [f'InkML Visualization: {file_path.name}']
        if show_latex and latex:
            # Loại bỏ ký tự $ và các ký tự có thể gây lỗi
            latex_display = latex.strip('$').strip()
            # Thay thế một số ký tự để hiển thị rõ hơn
            latex_display = latex_display.replace('&amp;', '&')
            latex_display = latex_display.replace('\\\\', ' \\\\ ')
            title_lines.append(f'Ground Truth: {latex_display}')
        
        # Sử dụng text thay vì suptitle để tránh lỗi render LaTeX
        # Đặt tiêu đề bằng cách sử dụng text annotation
        fig.text(0.5, 0.95, '\n'.join(title_lines), 
                ha='center', va='top', fontsize=12, 
                transform=fig.transFigure)
        
        ax.set_xlabel('X coordinate', fontsize=10)
        ax.set_ylabel('Y coordinate', fontsize=10)
        
        # Hiển thị thông tin về số lượng traces
        info_text = f'Total traces: {len(traces)}'
        if segments:
            unique_symbols = set(seg['symbol'] for seg in segments.values())
            info_text += f'\nSymbols: {len(unique_symbols)}'
        
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes, 
                fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        # Legend nếu có segmentation
        if show_segments and segments:
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), 
                     fontsize=9, framealpha=0.9)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Lỗi khi visualize: {str(e)}")
        print(f"Chi tiết lỗi: {type(e).__name__}")

# Hàm phụ để lấy thông tin chi tiết về file InkML
def get_inkml_info(file_path: str):
    """
    Lấy thông tin chi tiết về file InkML
    """
    file_path = Path(file_path)
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    info = {
        'filename': file_path.name,
        'traces_count': 0,
        'latex': None,
        'writer_info': {},
        'annotations': []
    }
    
    # Đếm số traces
    traces = root.findall('.//{http://www.w3.org/2003/InkML}trace')
    info['traces_count'] = len(traces)
    
    # Lấy tất cả annotations
    for ann in root.findall('.//{http://www.w3.org/2003/InkML}annotation'):
        ann_type = ann.get('type', 'unknown')
        ann_text = ann.text.strip() if ann.text else ''
        info['annotations'].append({'type': ann_type, 'text': ann_text})
        
        if ann_type == 'truth':
            info['latex'] = ann_text
    
    # Lấy thông tin writer nếu có
    for ann_xml in root.findall('.//{http://www.w3.org/2003/InkML}annotationXML'):
        if 'writer' in ann_xml.get('type', ''):
            for child in ann_xml:
                tag = child.tag.split('}')[-1]  # Lấy tên tag không có namespace
                info['writer_info'][tag] = child.text
    
    return info

# --------------------------------------------------
# Ví dụ sử dụng
if __name__ == '__main__':
    # Đường dẫn file cần visualize
    inkml_file = r'D:/code/projects/mathematical_formula_classification/handwritten-mathematical-expressions/versions/1/MatricesTest2014/RIT_MatrixTest_2014_2.inkml'
    
    # Hiển thị thông tin chi tiết
    print("=== THÔNG TIN FILE INKML ===")
    info = get_inkml_info(inkml_file)
    print(f"File: {info['filename']}")
    print(f"Số nét vẽ: {info['traces_count']}")
    print(f"LaTeX: {info['latex']}")
    print(f"Annotations: {len(info['annotations'])}")
    for ann in info['annotations']:
        print(f"  - {ann['type']}: {ann['text'][:50]}...")
    
    # Visualize
    print("\n=== VISUALIZING ===")
    visualize_inkml(inkml_file)

=== THÔNG TIN FILE INKML ===
File: RIT_MatrixTest_2014_2.inkml
Số nét vẽ: 8
LaTeX: )
Annotations: 13
  - truth: $ \begin {pmatrix} - 1 & 28 \\ 2 & 1 \end {pmatrix...
  - writer: annot_0...
  - UI: CROHME_2014_MATRICE_RIT_2...
  - copyright: RIT_2014...
  - truth: Segmentation...
  - truth: (...
  - truth: -...
  - truth: 1...
  - truth: 2...
  - truth: 2...
  - truth: 8...
  - truth: 1...
  - truth: )...

=== VISUALIZING ===


ValueError: 
 \begin {pmatrix} - 1 & 28 \\ 2 & 1 \end {pmatrix} 
 ^
ParseFatalException: Unknown symbol: \begin, found '\'  (at char 1), (line:1, col:2)

Error in callback <function _draw_all_if_interactive at 0x00000247160489D0> (for post_execute), with arguments args (),kwargs {}:


ValueError: 
 \begin {pmatrix} - 1 & 28 \\ 2 & 1 \end {pmatrix} 
 ^
ParseFatalException: Unknown symbol: \begin, found '\'  (at char 1), (line:1, col:2)

ValueError: 
 \begin {pmatrix} - 1 & 28 \\ 2 & 1 \end {pmatrix} 
 ^
ParseFatalException: Unknown symbol: \begin, found '\'  (at char 1), (line:1, col:2)

<Figure size 800x600 with 1 Axes>

In [ ]:
# Demo: Đọc và hiển thị một số file InkML từ dataset
import os

# Đường dẫn đến thư mục dataset  
base_path = r'D:/code/projects/mathematical_formula_classification/handwritten-mathematical-expressions/versions/1'

# Ví dụ 1: File từ MatricesTest2014
print("=== VÍ DỤ 1: MA TRẬN ===")
matrix_file = os.path.join(base_path, 'MatricesTest2014/RIT_MatrixTest_2014_2.inkml')
if os.path.exists(matrix_file):
    info = get_inkml_info(matrix_file)
    print(f"File: {info['filename']}")
    print(f"LaTeX: {info['latex']}")
    print(f"Số nét vẽ: {info['traces_count']}")
    visualize_inkml(matrix_file)
else:
    print(f"Không tìm thấy file: {matrix_file}")
